In [3]:
import scipy.optimize as so
import numpy as np
import h5py as h5
    

In [4]:
def UFid(U,V,Dim):
#    return np.sum(abs(U-V)**2)
    return abs(np.trace(np.matmul(U.conj().T,V))/Dim)**2

In [5]:
def QuantumFourier(Dim): 
    pi=np.arccos(-1.)
    x=np.array(range(Dim))
    return np.exp( 1j*2*pi*np.reshape( np.kron(x,x), [Dim,Dim] )/Dim )/np.sqrt(Dim)

In [6]:
def RandomUnitary(Dim):
    return np.linalg.qr( np.random.randn(Dim,Dim)+1j*np.random.randn(Dim,Dim) )[0]

In [7]:
def UFunction(Phase,N,F,Dim):
    U=np.eye(Dim)
    for k in range(N):
        U= np.exp( 1j*np.append([0] ,Phase[k*(Dim-1):(k+1)*(Dim-1)] ) )[:,np.newaxis]*U
        U=np.matmul(F,U)
    return np.exp(1j*np.append( [0], Phase[N*(Dim-1):(N+1)*Dim-1]))[:,np.newaxis]*U

In [8]:
def PUMIOpt(F,Dim,N,U):
    Seed = np.ones( (N+1)*(Dim-1) )
    TargetOpt = lambda x: 1-UFid( U , UFunction( x , N , F , Dim ) , Dim )
    argsmin = {"method": "L-BFGS-B"}
    Res=so.basinhopping(TargetOpt , Seed,minimizer_kwargs=argsmin,niter=5*10**2,disp=True)
    return Res.fun

In [9]:
def main():
    Repe=100
    Dim=4
    N=Dim
    
    F = QuantumFourier(Dim)
    file=h5.File("FFdataDim=4.h5","r") # Cambiar Nombre archivo 
    UU=file["re"][:,:,:].T+1j*file["im"][:,:,:].T
    
    ind2=file["Index"][:]-1
    MC=len(ind2)
    file.close()
    Data=np.zeros([MC,Repe])
    l=np.where(Data[:,0]<1)
    print(Dim,"-----",MC)
    for j in range(Repe):
        for i in range(MC):
            Data[i,j]=PUMIOpt( F,Dim,N,UU[:,:,ind2[ l[0][i] ]] )
        l2=np.where(Data[l,j]<10**(-10))
        for h in range(np.size(l2[0])):
            Data[l2[0][h],:]=Data[l2[0][h],j]
        l=np.where(Data[:,j]>10**(-10))
        MC=np.size(l[0])
        print(j,"-----",np.size(l[0]))

    file=h5.File("FFdataDim=4py.h5","w") #Cambiar nombre al archivo de salida
    file["infi"]=Data
    file["index1"]=l[0]
    file["index2"]=l2[0]
    file.close()

In [25]:

file=h5.File(".\data\HaarDistributed_Dim=3_Lay=Dim+0.h5","r")
UU=file["re"][:,:,:].T+1j*file["im"][:,:,:].T
ind2=file["Index"][:]
MC=len(ind2)
file.close()

KeyError: "Unable to synchronously open object (object 're' doesn't exist)"

In [ ]:
Dim=4
N=Dim
F=QuantumFourier(Dim)

In [11]:
u=UU[:,:,ind2]
PUMIOpt( F,Dim,N,u[:,:,1] )

NameError: name 'UU' is not defined

In [12]:
import sys

In [ ]:
sys.float_info.epsilon

2.220446049250313e-16